# Extended Graham scan algorithm

Use the cell below for all python code needed to realise the extended Graham scan algorithm (including any auxiliary data structures and functions you might need). The `extendedgrahamscan()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points that lie on the convex hull (`outputSet`).

In [ ]:
# Looking for the point with the lowest y-coordinate. If more than 1 point found, take the one with the lowest x-coordinate.
def find_start(points):
    start_point = points[0]
    for point in points:
        if point[1] < start_point[1]:
            start_point = point
        elif point[1] == start_point[1] and point[0] < start_point[0]:
            start_point = point
    return start_point

# Function returning tuple of cos value of an angle between start point and a given point and distance between these points 
def calculate_cos_dist(start_point, point):
    hyp = calculate_dist(start_point, point)
    return (-((point[0] - start_point[0])/hyp), hyp)

# Function returing a distance between the start point and a given point
def calculate_dist(start_point, point):
    return ((point[0] - start_point[0]) ** 2 + (point[1] - start_point[1]) ** 2) ** (1/2)

# Function finding up to 8 corner points and returns a list. Index explanation:
# 0) min x
# 1) biggest y - x
# 2) max y
# 3) biggest x + y
# 4) max x
# 5) lowest y - x
# 6) min y
# 7) lowest x + y
# Point can be both for example min x and min y
def find_corners(points):
    corners = [points[0] for x in range(8)]

    for x, y in points:
        if x < corners[0][0]:
            corners[0] = [x, y]
        if y > corners[2][1]:
            corners[2] = [x, y]
        if x > corners[4][0]:
            corners[4] = [x, y]
        if y < corners[6][1]:
            corners[6] = [x, y]

        if x + y < corners[7][0] + corners[7][1]:
            corners[7] = [x, y]
        if x + y > corners[3][0] + corners[3][1]:
            corners[3] = [x, y]
        if -x + y < -corners[5][0] + corners[5][1]:
            corners[5] = [x, y]
        if -x + y > -corners[1][0] + corners[1][1]:
            corners[1] = [x, y]

    return corners

# calculate linear coefficients of lines created by consecutive corners. 
# If corners are represented by the same point, (e.g. point is simultaneously min x and min y), return None. 
# If points create a perfect vertical line, append only their x coordinate.
# Else, append [a, b], where a and b are coefficients from the linear equation y = ax - b

def calculate_lines(corners):
    corners_len = len(corners)
    eq_list = []
    for i in range(corners_len):
        if corners[i] == corners[(i+1) % corners_len]:
            eq_list.append(None)
            continue
        if corners[i][0] - corners[(i+1) % corners_len][0] == 0:
            eq_list.append([corners[i][0]])
        else: 
            eq_list.append([(corners[i][1] - corners[(i+1) % corners_len][1])/(corners[i][0] - corners[(i+1) % corners_len][0]),\
                corners[i][1] - ((corners[i][1] - corners[(i+1) % corners_len][1])/(corners[i][0] - corners[(i+1) % corners_len][0])*corners[i][0])])

    return eq_list

# For every line check if point is over it or under it. If line do not exisits skip it. If its vertical compare just x values.
# Important! For certain lines point must be under then and for others it must be above them. This is the reason for so many lines of code here.
def not_in_octagon(lines, point):
    for line in lines[:2]:
        if line == None:
            continue

        if len(line) == 1:
            if point[0] <= line[0]:
                return True
        else:
            if line[0] * point[0] + line[1] <= point[1]:
                return True

    for line in lines[2:4]:
        if line == None:
            continue

        if len(line) == 1:
            if point[0] >= line[0]:
                return True
        else:
            if line[0] * point[0] + line[1] <= point[1]:
                return True
                        
    for line in lines[4:6]:
        if line == None:
            continue

        if len(line) == 1:
            if point[0] >= line[0]:
                return True
        else:
            if line[0] * point[0] + line[1] >= point[1]:
                return True

    for line in lines[6:]:
        if line == None:
            continue

        if len(line) == 1:
            if point[0] <= line[0]:
                return True
        else:
            if line[0] * point[0] + line[1] >= point[1]:
                return True

    return False


# Function returning cross product of two vectors (p1p2 x p1p3)
def cross_product(p1, p2, p3):
    return (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0])


# Standard Graham scan of already sorted points
def graham(start_point, points):
    stack = []
    stack.append(start_point)

    for point in points:
        while len(stack) > 1 and cross_product(stack[-2], stack[-1], point) <= 0:
            stack.pop()
        stack.append(point)
    return stack

# Encapsulating function taking inputSet of points, calculating the first point, preprocessing points, sorting points and running graham scan on them
def extendedgrahamscan(inputSet):  
    # Preprocessing
    corners = find_corners(inputSet)
    lines = calculate_lines(corners)
    inputSet = list(filter(lambda x: not_in_octagon(lines, x), inputSet))

    # Standard Graham Scan
    start_point = find_start(inputSet)
    inputSet.remove(start_point)
    inputSet.sort(key = lambda p: calculate_cos_dist(start_point, p))
    outputSet = graham(start_point, inputSet)

    return outputSet

Use the cell below for all python code needed to generate test data points (both random and those representing worst-case scenario).

In [ ]:
import random

#code for random data generation
def generate_positive_int_points(num_of_points, x_max, y_max):
    point_set = set()
    while len(point_set) != num_of_points:
        point = (random.randint(0,x_max), random.randint(0,y_max))
        point_set.add(point)
    
    point_list = []
    for x,y in point_set:
        point_list.append([x,y])

    return list(point_list)

#code for worst case data generation

# All points are collinear
def generate_positive_int_points_collinear(num_of_points, x_max):
    point_set = set()
    while len(point_set) != num_of_points:
        point = (random.randint(0,x_max), random.randint(1000, 1000))
        point_set.add(point)
    
    point_list = []
    for x,y in point_set:
        point_list.append([x,y])

    return list(point_list)




Use the cell below for all python code needed to test the `extendedgrahamscan()` function on the data generated above.

In [ ]:
import timeit

# test code average case
elements = [100, 500, 1000, 5000, 10000, 15000, 20000] # List of numbers of points
time_graham = [] # List for time results
it_num = 100 # Numbers of iterations for every number of points

for x in elements:    
    internal_time_list = []

    for _ in range(it_num): # For every number of points run graham it_num times with a different random inputset
        points = generate_positive_int_points(x, 32767, 32767) # Generate points
        start_time = timeit.default_timer()
        extendedgrahamscan(points) # Run graham scan
        end_time = timeit.default_timer()
        internal_time_list.append(end_time - start_time) # Adding time of each iteration to the list

    time_graham.append(sum(internal_time_list) / it_num) # Calculating the average time of all iterations

    print(x, time_graham[-1])

print(elements)
print(time_graham)

In [ ]:

# test code words case
elements = [100, 500, 1000, 5000, 10000, 15000, 20000] # List of numbers of points
time_graham = [] # List for time results
it_num = 100 # Numbers of iterations for every number of points

for x in elements:    
    internal_time_list = []

    for _ in range(it_num): # For every number of points run graham it_num times with a different random inputset
        points = generate_positive_int_points_collinear(x, 32767) # Generate points
        start_time = timeit.default_timer()
        extendedgrahamscan(points) # Run graham scan
        end_time = timeit.default_timer()
        internal_time_list.append(end_time - start_time) # Adding time of each iteration to the list

    time_graham.append(sum(internal_time_list) / it_num) # Calculating the average time of all iterations

    print(x, time_graham[-1])

print(elements)
print(time_graham)

*Optional*: Feel free to use the code below on small datasets (e.g., N = 10) to visually inspect whether the algorithm has been implemented correctly. The fragment below assumes both `inputSet` and `outputSet` to be lists of 2D points, with each point being a list of 2 elements (e.g., `[[x1,y1], [x2,y2], ..., [x_k,y_k]]`)

In [ ]:
import matplotlib.pyplot as plt

# inputSet and outputSet should have been defined above. 
# uncomment the next two lines only if you wish to test the plotting code before coding your algorithm

# inputSet = [[1,1], [2,2] , [3, 3], [4,4], [1,4], [3,1], [1, 5], [2, 4], [3, 5]]
# outputSet = [[1,1], [3,1] , [4, 4], [3,5], [1,5]]
inputSet = generate_positive_int_points(200, 32767, 32767)
plt.figure()

#first do a scatter plot of the inputSet
input_xs, input_ys = zip(*inputSet)
plt.scatter(input_xs, input_ys)
outputSet = extendedgrahamscan(inputSet)

#then do a polygon plot of the computed covex hull
outputSet.append(outputSet[0]) #first create a 'closed loop' by adding the first point at the end of the list
output_xs, output_ys = zip(*outputSet) 
plt.plot(output_xs, output_ys) 

plt.show() 